<a href="https://colab.research.google.com/github/LSSTDESC/DifferentiableHOS/blob/main/notebooks/spectrum_for_cosmology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0,'/Users/dl264294/Desktop/github/flowpm/')
sys.path.insert(0,'/Users/dl264294/Desktop/github/DifferentiableHOS/')
#from flowpm.spectrum_for_cosmology import power_spectrum_for_cosmology
from flowpm.tfbackground import z2a,a2z,rad_comoving_distance, a_of_chi
from flowpm.cosmo_Plank15 import cosmo_15
from flowpm.tfpower import linear_matter_power
from DifferentiableHOS.angular_power_tf import measure_power_spectrum_tf
from flowpm.tfpower import linear_matter_power
from flowpm.raytracing import  lightcone, Born
import flowpm 


In [2]:
nc=[32,32,320]   # size of the cube, number of cells
plane_size=32                    # number of pixel for x and  y 
Boxsize=[200,200,2000]          # Physical size of the cube
field=5.
n_steps=10

In [3]:
@tf.function
def power_spectrum_for_cosmology(
              Omega0_m,
              sigma8):
    cosmology=cosmo_15
    cosmology['Omega0_m']=tf.convert_to_tensor(Omega0_m,dtype=tf.float32)
    cosmology['sigma8']=tf.convert_to_tensor(sigma8,dtype=tf.float32)
    a_s=z2a(1.00)
    r = np.linspace(0,2000,10)
    a=a_of_chi(cosmology,r)
    ds=rad_comoving_distance(cosmology,a_s)
    init_stages = tf.linspace(0.1, a[-1], 4)
    initial_conditions = flowpm.linear_field(nc,    
                                            Boxsize, 
                                             lambda k: tf.cast(linear_matter_power(cosmology, k), tf.complex64),         
                                             batch_size=1)
    # Sample particles
    state = flowpm.lpt_init(initial_conditions, 0.1,cosmology)   
    # Evolve particles down to z=0
    final_state = flowpm.nbody(state, init_stages, nc, cosmology)         
    # Retrieve final density field
    state, lps_a, lps=lightcone(final_state, a[::-1], 
                                  nc, 
                                    field*60/plane_size, plane_size,
              cosmology)
    k_map=Born(lps_a,lps,ds,nc,Boxsize,plane_size,field,cosmology)
    k_map=tf.cast(k_map,dtype=tf.complex64)
    ell, power_spectrum=measure_power_spectrum_tf(k_map,field,plane_size)
    return ell, power_spectrum


In [ ]:
ell, power_spectrum =power_spectrum_for_cosmology(0.3075,0.8159)

In [4]:
Omega0_m= tf.constant(0.3075)
sigma8=tf.constant(0.8159)
with tf.GradientTape() as tape:
    tape.watch(Omega0_m)
    ell, power_spectrum= power_spectrum_for_cosmology(
              Omega0_m,
              sigma8)
dy_dx = tape.gradient(power_spectrum, Omega0_m)
print(dy_dx)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


InvalidArgumentError:  Input matrix is not invertible.
	 [[{{node gradients/LinearField/PartitionedCall_grad/PartitionedCall/gradients/IdentityN_grad/bdfGradients/while/body/_1446/gradients/IdentityN_grad/bdfGradients/while/bdf/while/body/_5001/gradients/IdentityN_grad/bdfGradients/while/bdf/while/while/body/_8439/gradients/IdentityN_grad/bdfGradients/while/bdf/while/while/while/body/_11685/gradients/IdentityN_grad/bdfGradients/while/bdf/while/while/while/while/body/_14164/gradients/IdentityN_grad/bdfGradients/while/bdf/while/while/while/while/triangular_solve/MatrixTriangularSolve}}]] [Op:__inference___backward_power_spectrum_for_cosmology_53470_159580]

Function call stack:
__backward_power_spectrum_for_cosmology_53470


In [ ]:

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_contours(fisher, pos,  nstd=1., ax=None, **kwargs):
    """
    Plot 2D parameter contours given a Hessian matrix of the likelihood
    """

    def eigsorted(cov):
    vals, vecs = linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:, order]

    mat = fisher
    cov = np.linalg.inv(mat)
    sigma_marg = lambda i: np.sqrt(cov[i, i])

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * sqrt(vals)
    ellip = Ellipse(xy=pos, width=width,
                  height=height, angle=theta, **kwargs)

    ax.add_artist(ellip)
    sz = max(width, height)
    s1 = 1.5*nstd*sigma_marg(0)
    s2 = 1.5*nstd*sigma_marg(1)
    ax.set_xlim(pos[0] - s1, pos[0] + s1)
    ax.set_ylim(pos[1] - s2, pos[1] + s2)
    plt.draw()
    return ellip

In [ ]:
%pylab inline
import jax
import jax_cosmo as jc
import jax.numpy as np

In [ ]:
cosmo_jax = jc.Planck15()
chi_jax = np.linspace(0, 2000,10)


In [ ]:
params=tf.convert_to_tensor([Omega0_m,sigma8])

In [ ]:
z = linspace(0,2,100)
pz = zeros_like(z)
pz[50] =1. 
nzs_s=jc.redshift.kde_nz(z, pz, bw=0.05)
# let's draw the nz on a new array of redshifts
zsamp = np.linspace(0,2,128)
plot(zsamp, nzs_s(zsamp))
nzs = [nzs_s]
probes = [ jc.probes.WeakLensing(nzs, sigma_e=0.26) ]
ell = np.logspace(1,4) # Defines a range of \ell
# And compute the data vector
cls = jc.angular_cl.angular_cl(cosmo_jax, ell, probes)

# we compute the covariance matrix using the tools from the Jax module
mu, cov = jc.angular_cl.gaussian_cl_covariance_and_mean(cosmo_jax, ell, probes, sparse=True);

In [ ]:
# Now we can compose the Fisher matrix:
F_2 = jc.sparse.dot(dy_dx.T, jc.sparse.inv(cov), dy_dx)
plot_contours(F, params, fill=False,color='black',lw=4);
xlabel('Omega_m')
ylabel('sigma8');